# Create the sentiment module from the classifiers that were trained with labelled old tweets

In [1]:
#we are using nltk libraries with scikit to do most of the natural language processing
import nltk
import random
from nltk.classify.scikitlearn import SklearnClassifier
import pickle
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.classify import ClassifierI
from statistics import mode
from nltk.tokenize import word_tokenize

#the vote classifier will compare all the classifiers we are using, which is then used to vote to determine which algorithm is better suited

class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)

    def sent_confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf

    
#On the initially trained model, we are using PICKLE modules to save them for easy retrieval. The pickle module implements binary protocols for serializing and de-serializing a Python object structure. “Pickling” is the process whereby a Python object hierarchy is converted into a byte stream, and “unpickling” is the inverse operation    

documents_f = open("movie_reviews/pickledfiles/documents.pickle", "rb")
documents = pickle.load(documents_f)
documents_f.close()


#word_features consists of all the features that were retrieved from the intial files containing positive and negative tweets that were loaded to train the multiple classifiers

word_features5k_f = open("movie_reviews/pickledfiles/word_features5k.pickle", "rb")
word_features = pickle.load(word_features5k_f)
word_features5k_f.close()


def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

#contains all the features sets that were creted early and now we are 'pickling' them for easy retrieval 

featuresets_f = open("movie_reviews/pickledfiles/featuresets.pickle", "rb")
featuresets = pickle.load(featuresets_f)
featuresets_f.close()

random.shuffle(featuresets)
print(len(featuresets))

testing_set = featuresets[10000:]
training_set = featuresets[:10000]



open_file = open("movie_reviews/pickledfiles/originalnaivebayes5k.pickle", "rb")
classifier = pickle.load(open_file)
open_file.close()


open_file = open("movie_reviews/pickledfiles/MNB_classifier5k.pickle", "rb")
MNB_classifier = pickle.load(open_file)
open_file.close()



open_file = open("movie_reviews/pickledfiles/BernoulliNB_classifier5k.pickle", "rb")
BernoulliNB_classifier = pickle.load(open_file)
open_file.close()


open_file = open("movie_reviews/pickledfiles/LogisticRegression_classifier5k.pickle", "rb")
LogisticRegression_classifier = pickle.load(open_file)
open_file.close()


open_file = open("movie_reviews/pickledfiles/LinearSVC_classifier5k.pickle", "rb")
LinearSVC_classifier = pickle.load(open_file)
open_file.close()


open_file = open("movie_reviews/pickledfiles/SGDC_classifier5k.pickle", "rb")
SGDC_classifier = pickle.load(open_file)
open_file.close()




voted_classifier = VoteClassifier(
                                  classifier,
                                  LinearSVC_classifier,
                                  MNB_classifier,
                                  BernoulliNB_classifier,
                                  LogisticRegression_classifier)




def sentiment(text):
    feats = find_features(text)
    return voted_classifier.classify(feats),voted_classifier.confidence(feats)

10664


## Section for streaming tweets from twitter and saving them into a mySQLdb for easy retrieval and processing

In [20]:
import sentimentmodule as SM
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import MySQLdb
import json
import re
import string
import time
from unidecode import unidecode



# create mySQL connection to the local host using MySQLdb module.We are setting the charset to utf8mb4 to deal with smileys, emoticons, foriegn characters etc
conn = MySQLdb.connect("localhost","root","mongo1234","mydb1",use_unicode=True, charset="utf8mb4")

c = conn.cursor()


#consumer key, consumer secret, access token, access secret which is passed to the oauth for tweepy.
ckey="nu4okkirMdCPQBUbsph7M4nOo"
csecret="iN8iNWOayRRJpEOhH9TmprIUO9jPIz511TeFDLTyZ6WOMuQ51x"
atoken="1068606021185282050-VjaqxC0OA21j5KUtMwohBuB9O9rV7o"
asecret="mUoPu2a5lIchq6p1XTOi9T7ohjqELbpCKP5MarLNQzhG0"


# this is our SQL for adding the tweetID and results into our DB
add_tweet = ("INSERT INTO tweets3"
             "(tweetID, movies_movieName, dateTime, tweet, result, confidence)"
             "VALUES (%s, %s, %s, %s, %s, %s)")


# Twitter's results just give us back the tweet, but don't tell us which keyword it was found with
# so, we have to use a keyword dictionary to search the tweet and match it back up to the movie
movieKeyword = dict()
movieKeyword['aquaman'] = 'Aquaman'
movieKeyword['bumblebee'] = 'Bumblebee'
movieKeyword['crimsonpeak'] = 'Crimson Peak'
movieKeyword['marypoppinsreturns'] = 'Mary Poppins Returns'
movieKeyword['marypoppins'] = 'Mary Poppins Returns'
movieKeyword['poppins'] = 'Mary Poppins Returns'
movieKeyword['vice'] = 'Vice'
movieKeyword['vicemovie'] = 'Vice'
movieKeyword['spiderman'] = 'Spiderman into the Spider-Verse'
movieKeyword['spiderverse'] = 'Spiderman into the Spider-Verse'
movieKeyword['glassmovie'] = 'Glass'
movieKeyword['captainmarvel'] = 'Captain Marvel'
movieKeyword['mortalengines'] = 'Mortal Engines'
movieKeyword['onceuponadeadpool'] = 'Once Upon a Deadpool'
movieKeyword['jokermovie'] = 'Joker'
movieKeyword['joker'] = 'Joker'
movieKeyword['avengersendgame'] = 'Avengers Endgame'
movieKeyword['avengers'] = 'Avengers Endgame'
movieKeyword['endgame'] = 'Avengers Endgame'
movieKeyword['greenbook'] = 'Greenbook'
movieKeyword['welcometomarven'] = 'Welcome to Marwen'
movieKeyword['shazam'] = 'Shazam'
movieKeyword['themule'] = 'The Mule'




#exclude punctuations
regex = re.compile('[%s]' % re.escape(string.punctuation))

#build the class used to process tweets to check for feeds
class twitter_streaming(StreamListener):

    def on_data(self, data):
        all_data = json.loads(data)
        if 'text' in all_data:
            tweet = all_data['text']
            tweet = unidecode(tweet)
            tweetNoPunctuation = regex.sub('', tweet)
#we want to make sure while compiling tweets, we do not include the oens that are retweeted
            if not all_data['retweeted'] and not tweet.startswith('RT') and 't.co' not in tweet:
                sentiment_value, confidence = SM.sentiment(tweetNoPunctuation)
                print(tweet, sentiment_value, confidence)

                found = False
                movie_name = ""
                for word in tweetNoPunctuation.split(" "):  
                    if word.lower() in movieKeyword.keys():
                        movie_name = movieKeyword[word.lower()]
                        print("Found keyword: ", word, " belongs to movie: ", movie_name)
                        found = True
                        break

                if found:
                    created_at = time.strftime('%Y-%m-%d %H:%M:%S')  
                    #twitter JSON is being parsed with queries below and using sentiment module, we are assigning confidence values
                    tweet_data = (all_data['id_str'], movie_name, created_at, tweet, sentiment_value.lower(), confidence)
                    c.execute(add_tweet, tweet_data)
                    conn.commit()
                    

        return True
#error handling, since tweepy tends to time out with twitter with out any reason closing the connection from their side
    def on_limit(self, track):
        print('Limit hit! Track = %s' % track)
        return True

    def on_error(self, status):
        print(status)

    def on_disconnect(self, notice):
        print(notice)


# Twitter Authorization path
auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

# create twitter stream, which in turn will start streaming tweeets in JSON format, which we are using to query its metadata and store them separately on to the database
twitterStream = Stream(auth, twitter_streaming())

# Movie keywords  to search for in tweets
twitterStream.filter(track=["aquaman", "bumblebee", "crimsonpeak", "mary", "poppins","marypoppins", "vice", "vicemovie" "captainmarvel", "jokermovie", "joker", "mortalengines", "spiderverse", "spiderman","marypoppinsreturns", "avengersendgame", "avengers", "endgame", "themule", "glassmovie", "shazam", "onceuponadeadpool" ], languages=["en"])


cursor.close()
conn.close()


PS2k each not PS200k Mary, sadly  #Gogglebox neg 0.8
Y'all are trolling us. Not falling for this neg 0.8
Chris's favourite neg 1.0
IT'S CALLED ENDGAME LIKE BITCH I'M SHAKING neg 0.8
------------------Found keyword:  ENDGAME  belongs to movie:  Avengers Endgame
lmfao is this shrek pos 0.8
so whose comin? neg 0.8
Reading this thread made me lose all hope in today's society neg 0.8
tony wont run out of oxygen because i will launch myself into deep space personally to give him my own oxygen #AvengersEndgame pos 0.8
------------------Found keyword:  AvengersEndgame  belongs to movie:  Avengers Endgame
what................... neg 0.8
If you have kids and/or nieces and nephews, you should take them to see this. ;) neg 0.8
@LuyiDee Great! Just thought I'd say hi. Nice seeing you the other day. So avengers abi? neg 1.0
------------------Found keyword:  avengers  belongs to movie:  Avengers Endgame
@9GAG Avengers: Hero of The Half Truth. Hahaha  #TeamThanos neg 0.8
------------------Found keywor

If you get a text from some lady named Lizi saying I gave her your number just ignore her (she's trying to sell you some Mary Kay) neg 0.8
sorry... but i got chills neg 1.0
James Charles Flasback Mary drama: neg 0.8
THEY BETTER NOT SLEEP ON HIM neg 0.8
I've only seen the trailer to #AvengersEndgame and I know I'm gonna bawl my eyes out #Avengers4 neg 1.0
------------------Found keyword:  AvengersEndgame  belongs to movie:  Avengers Endgame
It me.  neg 0.8
SHUT. UP neg 0.8
Fuck, the accuracy neg 0.8
Hot take: no one, including Cap, dies. #AvengersEndgame neg 0.8
------------------Found keyword:  AvengersEndgame  belongs to movie:  Avengers Endgame
The official title isn't going to be Avengers End Game  pos 1.0
------------------Found keyword:  Avengers  belongs to movie:  Avengers Endgame
y'all are unnecessarily problematic and it honestly hurts my head. neg 0.8
Can't wait  neg 0.8
He shaves his beard because he's going back in time. He didn't have a beard at the first avengers movie. p

Smash

Avengers Endgame

Cohen Memo 

Could this day get any more insane? pos 1.0
him and spiderman about the same spiderman may be stronger. neg 1.0
------------------Found keyword:  spiderman  belongs to movie:  Spiderman into the Spider-Verse
It's funny just thinking that there's probably a nigga who made a joker  smash ultimate discord pos 1.0
------------------Found keyword:  joker  belongs to movie:  Joker
I can't  neg 0.8
i will never not retweet out of context eric andre neg 1.0
@perepereden I'm super hyped about Joker and your hype is multiplying my hype exponentially. Hahaha neg 1.0
------------------Found keyword:  Joker  belongs to movie:  Joker
I AM NOT READY! 
#AvengersEndgame neg 0.8
IM GOING TO HAVE TO GO SEE THE NEW AVENGERS WITHOUT CHRISTIAN. NOOOOO neg 0.8
------------------Found keyword:  AVENGERS  belongs to movie:  Avengers Endgame
@VICE And according to @SouthPark neg 0.8
------------------Found keyword:  VICE  belongs to movie:  Vice
Congrats Mary neg 0.8
hi goo

This smells like it's gonna suck neg 1.0
Age of Ultron... neg 0.8
FRISBEE THROWER neg 0.8
woke up this morning to that new endgame trailer; it's gonna be a good weekend pos 0.6
------------------Found keyword:  endgame  belongs to movie:  Avengers Endgame
I just wanna say rn that if for some stupid reason Iron Man dies in Avengers I will NEVER heal neg 1.0
------------------Found keyword:  Avengers  belongs to movie:  Avengers Endgame
@EuniceMurt im scared. neg 0.8
@VICE Yeah...i'm worried about that. neg 0.8
------------------Found keyword:  VICE  belongs to movie:  Vice
Joker from persona 5 is DLC in smash bros. pos 0.8
------------------Found keyword:  Joker  belongs to movie:  Joker
@cinemasins That was a good one. I won't be surprised if that's true.  #AvengersEndgame pos 1.0
------------------Found keyword:  AvengersEndgame  belongs to movie:  Avengers Endgame
Just saw the new Avengers trailer and all I gotta say is that I'm ready for April neg 1.0
------------------Found keyword

Him and his sorcerer army chilling in the Astral Plane. PERIOD neg 0.8
IM NOT CRYING YOU'RE CRYING neg 0.8
@CaseyMattox_ @benshapiro Ben Shapiro president Donald Trump vice president hahahahah neg 0.8
------------------Found keyword:  vice  belongs to movie:  Vice
#MortalEngines looks dumb as fuck neg 1.0
------------------Found keyword:  MortalEngines  belongs to movie:  Mortal Engines
because we're endgame archie  neg 0.8
------------------Found keyword:  endgame  belongs to movie:  Avengers Endgame
I was hoping for a better avengers trailer neg 1.0
------------------Found keyword:  avengers  belongs to movie:  Avengers Endgame
oh mY god neg 1.0
CAN'T WAIT TO WATCH AVENGERS 2019  neg 0.8
------------------Found keyword:  AVENGERS  belongs to movie:  Avengers Endgame
Gaaaaddddd , it's here . Smack that , can't wait #AvengersEndgame neg 0.8
------------------Found keyword:  AvengersEndgame  belongs to movie:  Avengers Endgame
This new avengers trailer got me fucked up neg 0.8
---------

...

@Avengers my end neg 0.8
------------------Found keyword:  Avengers  belongs to movie:  Avengers Endgame
Damn this movie has taken forever neg 0.8
I'm so happy that you're back! You seem a little psychotic but that's fine with me pos 1.0
Asking the important questions pos 1.0
Take my money immediately neg 0.8
ok that was so fuckin DRAMATIC for no reason pos 1.0
Jack in Titanic neg 0.8
Holy fucking shit[?] neg 0.8
goodmorning to everyone but marvel cause i already know endgame is gonna fuck me up neg 1.0
------------------Found keyword:  endgame  belongs to movie:  Avengers Endgame
Spiderman enter the spider verse was amazing! Watch it in 3D when it's out, best use of 3D I've seen. Yaaaaaas pos 1.0
------------------Found keyword:  Spiderman  belongs to movie:  Spiderman into the Spider-Verse
I'm sad about how sad Tony looks in the Avengers teaser. Out of context he looks nearly suicidal. neg 0.8
------------------Found keyword:  Avengers  belongs to movie:  Avengers Endgame
IM GONNA TH

joker in smash and avengers trailer..... today was exhausting neg 0.8
------------------Found keyword:  joker  belongs to movie:  Joker
Avengers: Endgame is not a great title tbh but I don't care! I just want the damn movie! pos 1.0
------------------Found keyword:  Avengers  belongs to movie:  Avengers Endgame
Happy Birthday Mama Mary! neg 0.8
@Eva_Friedman Avengers: Call It What You Want... because we're all watching it regardless tbh neg 0.8
------------------Found keyword:  Avengers  belongs to movie:  Avengers Endgame
YAAASS BISSSSHH GIVE ME PEPPER IN ARMOR neg 0.8
TODAY IS THE FEAST OF IMMACULATE CONCEPTION OF MARY.  #NonWorkingDay
#Good #Morning neg 0.8
Fast forward to april pls :(( #AvengersEndGame neg 0.8
------------------Found keyword:  AvengersEndGame  belongs to movie:  Avengers Endgame
Avengers: Endgame trailer approaching 27 million views. It's been ~9 hours. That's ~3 million an hour. neg 0.8
------------------Found keyword:  Avengers  belongs to movie:  Avengers Endgam

him being naked? neg 0.8
@KALELAURENT After I saw Avengers trailer neg 0.8
------------------Found keyword:  Avengers  belongs to movie:  Avengers Endgame
@EvansTed101 But it should be vice versa neg 0.8
------------------Found keyword:  vice  belongs to movie:  Vice
@Avengers Wat de paking sit neg 1.0
------------------Found keyword:  Avengers  belongs to movie:  Avengers Endgame
This is why I didn't kill myself during finals season neg 0.8
new goal in life survive till April. neg 0.8
On my birthday! pos 1.0
This is genius neg 0.8
Me watching avengers 4 trailer , mana Spider-Man tak keluar pun neg 0.8
------------------Found keyword:  avengers  belongs to movie:  Avengers Endgame
We miss you Clint.. [?] neg 1.0
@TaylorSwitch64 Nah... But we got Persona 5's Joker neg 0.8
------------------Found keyword:  Joker  belongs to movie:  Joker
Yup. I'll take a bloody mary right now. neg 1.0
@MarvelStudios MY LAWYERS WILL BE IN CONTACT neg 0.8
I've been thinking about this since I saw the trail

"When I drift off, I will dream about you. It's always you." #AvengersEndgame neg 0.8
------------------Found keyword:  AvengersEndgame  belongs to movie:  Avengers Endgame
I want to watch #SpiderVerse again it was funny pos 1.0
------------------Found keyword:  SpiderVerse  belongs to movie:  Spiderman into the Spider-Verse
@bbcquestiontime Gammony fool. We're leaving them not vice versa neg 0.8
------------------Found keyword:  vice  belongs to movie:  Vice
Casey is an angel and makes amazing pins that would make cute gifts or stocking stuffers pos 1.0
FUck Avengers 4, I'm still decoding Ant-Man &amp; The Wasp neg 0.8
------------------Found keyword:  Avengers  belongs to movie:  Avengers Endgame
Is this a new movie then or a softer version of one of the first two? pos 1.0
So if you wanna go on a movie date lmkkkk  neg 1.0
Marvel really made us watch Ant-Man and the Wasp to set up for Avengers neg 0.8
------------------Found keyword:  Avengers  belongs to movie:  Avengers Endgame
Thi

KeyboardInterrupt: 